### Step 4: Creating Soil Moisture Maps

Now we can implement our two wining models (FVS and BVS) in Google Earth Engine. This will be a straightfoward procedure because it is only an equation.

Let's start:

In [1]:
import ee
import geemap
ee.Initialize()

# Delta_area = ee.Geometry.Rectangle([-112.658355,39.442565,-112.606132,39.357404])
Delta_area=ee.Geometry.Point([-112.577170, 39.352178])
Map1 = geemap.Map() 
Map1.addLayer(Delta_area, {}, name='Delta_area')
Map1.centerObject(Delta_area, 12)

In [2]:
# processing landsat 7 image and needed spectral indices

import eemont

landsat7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filterBounds(Delta_area) # Landsat 7 archive.
landsat7 = landsat7.preprocess()
landsat7 = landsat7.spectralIndices(['DSWI3','NRFIr','CVI', 'DSWI4', 'ExG'])


Let's implement the equation for the soil moisture map:

let's read the fvs csv file

In [3]:
import csv

# opening the CSV file
with open('./Results Step 3/fvs_coefficients.csv', mode ='r') as file:

# reading the CSV file
    csvFile = csv.reader(file)

    # displaying the contents of the CSV file
    for lines in csvFile:
        print(lines)

['Intercept: ']
['6.91459367001865']
['Slopes:']
['-1.9616626960772763']
['1.8852295229490146']
['-6.042066857865608']
['0.26009658359502463']
['1.7012957676684124']


Implement the linear regression in an equation

In [4]:
def fvs_sm(image):
    # image=image.clip(Delta_area)
    SM=image.expression('6.91459367001865-1.9616626960772763*SR_B4+1.8852295229490146*SR_B5-6.042066857865608*(ST_B6/273.15)+0.26009658359502463*DSWI3+1.7012957676684124*NRFIr', {
        'SR_B4':image.select('SR_B4'),
        'SR_B5':image.select('SR_B5'),
        'ST_B6':image.select('ST_B6'),
        'DSWI3':image.select('DSWI3'),
        'NRFIr':image.select('NRFIr')}).rename('SM')
    return image.addBands(SM)

In [5]:
# Generating the Soil Moisture band 

landsat7_fvs=landsat7.map(fvs_sm)



Selecting and visualizing the Landsat images from the Step 1

In [6]:
landsat7_may_13_fvs = ee.Image(landsat7_fvs.filterDate('2012-05-12','2012-05-14').first()) # all commands in one line.
landsat7_may_29_fvs = ee.Image(landsat7_fvs.filterDate('2012-05-28','2012-05-30').first()) # all commands in one line.
landsat7_jun_14_fvs = ee.Image(landsat7_fvs.filterDate('2012-06-13','2012-06-15').first()) # all commands in one line.
landsat7_jun_30_fvs = ee.Image(landsat7_fvs.filterDate('2012-06-29','2012-07-01').first()) # all commands in one line.

In [7]:
vis_params = {
  'min': 0.0,
  'max': 0.6,
  'palette': ['orange','yellow','green','cyan','blue'],
}

legend_colors = ['orange','yellow','green','cyan','blue']

In [8]:
Map1.addLayer(landsat7_may_13_fvs.select('SM'), vis_params,'May13')
Map1.addLayer(landsat7_may_29_fvs.select('SM'), vis_params,'May29')
Map1.addLayer(landsat7_jun_14_fvs.select('SM'), vis_params,'Jun14')
Map1.addLayer(landsat7_jun_30_fvs.select('SM'), vis_params,'Jun60')

Map1.add_colorbar_branca(
    colors=legend_colors, vmin=0.1, vmax=0.6, categorical=False, step=6, layer_name="Soil Moisture"
)
Map1

Map(center=[39.352178, -112.57717], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

Now implementing the bvs equation

In [9]:
import csv

# opening the CSV file
with open('./Results Step 3/bvs_coefficients.csv', mode ='r') as file:

# reading the CSV file
    csvFile = csv.reader(file)

    # displaying the contents of the CSV file
    for lines in csvFile:
        print(lines)

['Intercept: ']
['8.441544078068858']
['Slopes:']
['-6.655060951978442']
['-0.16289132840524534']
['0.3772804497778246']
['-0.7520829377015303']
['-2.554829741032563']
['1.3900236923110967']


In [10]:
#'ST_B6', 'CVI', 'DSWI3', 'DSWI4', 'ExG', 'NRFIr'

def bvs_sm(image):
    # image=image.clip(Delta_area)
    SM=image.expression('8.441544078068858-6.655060951978442*(ST_B6/273.15)-0.16289132840524534*CVI+0.3772804497778246*DSWI3-0.7520829377015303*DSWI4-2.554829741032563*ExG+1.3900236923110967*NRFIr', {
        'ST_B6':image.select('ST_B6'),
        'CVI':image.select('CVI'),
        'DSWI3':image.select('DSWI3'),
        'DSWI4':image.select('DSWI4'),
        'ExG':image.select('ExG'),
        'NRFIr':image.select('NRFIr')}).rename('SM')
    return image.addBands(SM)

In [11]:
# Generating the Soil Moisture band 

landsat7_bvs=landsat7.map(bvs_sm)

Selecting and visualizing the Landsat images from the Step 1

In [12]:
landsat7_may_13_bvs = ee.Image(landsat7_bvs.filterDate('2012-05-12','2012-05-14').first()) # all commands in one line.
landsat7_may_29_bvs = ee.Image(landsat7_bvs.filterDate('2012-05-28','2012-05-30').first()) # all commands in one line.
landsat7_jun_14_bvs = ee.Image(landsat7_bvs.filterDate('2012-06-13','2012-06-15').first()) # all commands in one line.
landsat7_jun_30_bvs = ee.Image(landsat7_bvs.filterDate('2012-06-29','2012-07-01').first()) # all commands in one line.

In [13]:
vis_params = {
  'min': 0.0,
  'max': 0.6,
  'palette': ['orange','yellow','green','cyan','blue'],
}

legend_colors = ['orange','yellow','green','cyan','blue']

In [14]:
Map2 = geemap.Map() 
Map2.addLayer(Delta_area, {}, name='Delta_area')
Map2.centerObject(Delta_area, 12)

In [15]:
Map2.addLayer(landsat7_may_13_bvs.select('SM'), vis_params,'May13')
Map2.addLayer(landsat7_may_29_bvs.select('SM'), vis_params,'May29')
Map2.addLayer(landsat7_jun_14_bvs.select('SM'), vis_params,'Jun14')
Map2.addLayer(landsat7_jun_30_bvs.select('SM'), vis_params,'Jun60')

Map2.add_colorbar_branca(
    colors=legend_colors, vmin=0.1, vmax=0.6, categorical=False, step=6, layer_name="Soil Moisture"
)
Map2

Map(center=[39.352178, -112.57717], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBo…

Values of soil moisture from both, forward and backward variable selection are similar. This indicates:

    1) Multiple models can be derived to solve a function.
    2) The variable selection procedure can affect the selected inputs and the resulting model.
    3) A rich set of potential predictors can yield simple models that are robust.